In [1]:
import pandas as pd
import numpy as np
import pickle

import sys
sys.path.insert(0, "../src")
sys.path.insert(0, "../")
from d2v_recommender import *
from config import config
from src.processing import load_d2v_formated_data


In [2]:
recommender = D2V_Recommender(**config.d2v_params)

train = pd.read_csv(config.train_data_path)
test = pd.read_csv(config.test_data_path)
d2v_train = load_d2v_formated_data(config.d2v_train_data_path)

# # learn embeddings for rated users
recommender.fit_rated_embeddings(
    d2v_train, save_path=config.rated_embeddings_path
)

recommender.load_rated_vec(config.rated_embeddings_path)

# learn embeddings for raters as the mean of embeddings of those they matched with
recommender.fit_rater_embeddings(train, save_path=config.rater_embeddings_path)
recommender.load_rater_vec(config.rater_embeddings_path)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 94.28it/s]


,0
1,"[0.00040496275, -0.0006818027, -0.00033978047,..."
2,"[0.00024747473, 0.0005912651, -0.0017073135, 0..."
3,"[0.0009146712, 0.002318329, -9.348209e-06, 0.0..."
4,"[0.0033158925, -0.0013413565, 0.0013407932, -0..."
5,"[-0.00014638172, 0.0007400165, -0.0003619319, ..."
...,...
96,"[-0.0006265093, 0.002062257, -0.0006437324, 0...."
97,"[0.0004198415, -0.0040085176, -0.0012637483, -..."
98,"[0.00032986057, -0.00022659083, -0.0009847261,..."
99,"[-0.0003149905, 0.0009315111, 0.00068687455, -..."


In [8]:
recommender.prepare_X_y_dataset(train, test, data_dict_path=config.data_dict_path)

Train N=10905 - Test N=2779
Train data N=13684
After skipping unseen rated users: N=3455


In [4]:
train_ = train.copy()
test_ = test.copy()
train_["set"] = "train"
test_["set"] = "test"
data = pd.concat([train_, test_])  # we will ignore the index

print(f"Train N={len(train_)} - Test N={len(test_)}")
assert len(data) == (len(train_) + len(test_))

# TODO: could be even further vectorized
data["rater"] = data["rater"].apply(recommender.get_single_rater_vec)
data["rated"] = data["rated"].apply(recommender.get_single_rated_vec)
data["vec_delta"] = data["rater"] - data["rated"]  # piecewise array operations
data = data[["vec_delta", "m", "set"]]

Train N=10905 - Test N=2779


In [7]:
data.dropna()

,vec_delta,m,set
0,"[-0.0032073178, -0.0050240033, 0.0013888634, -...",0.0,train
1,"[0.00798453, -0.0097543765, -0.005961188, -0.0...",0.0,train
2,"[0.008539897, -0.0018408545, -0.0044120094, -0...",1.0,train
3,"[-0.0009370241, -0.0014134569, -0.0067626326, ...",0.0,train
4,"[0.0044528255, -0.00509703, -0.00012638792, -0...",1.0,train
...,...,...,...
2511,"[0.010319347, -0.0056337304, 0.0026717484, 0.0...",0.0,test
2526,"[0.0022081328, -0.008494749, 0.009288474, -0.0...",1.0,test
2562,"[-0.0021240958, -0.009882071, 0.0007747633, -0...",1.0,test
2594,"[0.0016053898, -0.008498546, -0.010645248, 0.0...",1.0,test
